In [2]:
import numpy as np

In [3]:
def FK(DH_Paramters):  
    DH = DH_Paramters
    HTM = np.zeros((4,4),dtype='i')
    HTM_List = []
    TTM = np.eye(4, dtype='i')
    Z = []
    Z.append(np.array([0,0,1]))
    t = []
    t.append(np.array([0,0,0]))
    for i in range(DH_Paramters.shape[0]):              
        HTM = np.array([[np.cos(DH[i,3]),-np.sin(DH[i,3])*np.cos(DH[i,1]), np.sin(DH[i,3])*np.sin(DH[i,1]), DH[i,0]*np.cos(DH[i,3])],
        [np.sin(DH[i,3]), np.cos(DH[i,3])*np.cos(DH[i,1]), -np.cos(DH[i,3])*np.sin(DH[i,1]), DH[i,0]*np.sin(DH[i,3])],
        [0, np.sin(DH[i,1]), np.cos(DH[i,1]), DH[i,2]],
        [0, 0, 0, 1]])
        HTM_List.append(HTM)
        TTM = np.matmul(TTM,HTM)
        Z.append(TTM[0:3,-2])
        t.append(TTM[0:3,-1])
    return TTM, Z, t


In [4]:
def Jacobian(dhParameters):
    Forward_Kinematics = FK(dhParameters)
    Z = Forward_Kinematics[1]
    t = Forward_Kinematics[2]
    J_v = []
    J_w = []
    for i in range(dhParameters.shape[0]):
        temp = Z[i]*(t[6]-t[i])
        J_v.append(temp)
    for j in range(dhParameters.shape[0]):
        J_w.append(Z[j])
    a_temp = np.zeros((6,1))
    for u in range(6):
        a = np.concatenate((J_v[u],J_w[u])).reshape(6,1)
        a_temp = np.concatenate((a_temp,a),axis = 1)
    Jacobian_matrix = a_temp[:,1::]
    return Jacobian_matrix

In [5]:
a = np.array([1,1,2,3,4,5])
d = np.array([2,1,2,3,4,5])
alpha = np.array([3,1,2,3,4,5])
q_offset = np.array([4,1,2,3,4,5])
dh_paras = np.array([a,d,alpha,q_offset]).T
dh_paras

array([[1, 2, 3, 4],
       [1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3],
       [4, 4, 4, 4],
       [5, 5, 5, 5]])

In [6]:
# Jacobian Matrix
J = Jacobian(dh_paras)
print('Jacobian Matrix: \n',J)

Jacobian Matrix: 
 [[ 0.         -3.58134347 -4.50176482 -2.94013137  2.1388401   1.6057675 ]
 [-0.         -1.79591158  1.16275799 -1.56697303  0.98175977  1.14738085]
 [ 3.56003347 -0.23305616 -0.13469302  1.97357176  1.23976344  2.24685165]
 [ 0.         -0.68815856 -0.69145383 -0.36483398  0.22997227  0.30240432]
 [ 0.          0.59435646 -0.33840934  0.34148913 -0.35001462 -0.28974152]
 [ 1.         -0.41614684 -0.638256    0.86618782 -0.90807628  0.9080757 ]]


In [7]:
# Determinant of the Jacobian
np.linalg.det(J)

-0.23759425349691318